In [ ]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# Extract csv into dataframe

The data source is a csv file related to the movement within communities as affected by the COVID-19 pandemic. This report from Google tracked movement trends by country over time, across multiple places such as: retail & recreation, groceries & pharmacies, parks, transit stations, workplaces, and residential.

In [ ]:
# import and read csv
mobility_file = "Resources/Global_Mobility_Report.csv"
mobility_df = pd.read_csv(mobility_file, dtype="unicode")
mobility_df.head()

# Transform dataframe

A series of steps are applied to the extracted data in order to prepare it for loading into the database.

From the extracted data, get the country-level mobility data only.

- convert date column from object to date.
- remove rows with dates other than year 2020.
- retain country-level data only by removing rows with values under col 2-6.
- delete col 0 & 2-7 (not needed).
- convert cols percent_change_from_baselin from object dtype to integer.
- rename columns.
- set index to country.

In [ ]:
# get number of rows in the df
print(len(mobility_df.index))

In [ ]:
# get all column names and datatypes
mobility_df.dtypes

In [ ]:
# convert date into datetime format
mobility_df['date'] = pd.to_datetime(mobility_df['date'])
mobility_df.info()

In [ ]:
# keep only 2020 mobility data
mobility20_df = mobility_df[mobility_df['date'].dt.year == 2020]
print(len(mobility20_df.index))

In [ ]:
# retain only country-level mobility data by removing rows with values under columns 2 to 6
countries_df = mobility20_df.loc[((mobility20_df['sub_region_1'].isna()) & (mobility20_df['sub_region_2'].isna()) & (mobility20_df['metro_area'].isna()) & (mobility20_df['iso_3166_2_code'].isna()) & (mobility20_df['census_fips_code'].isna())), :]
print(len(countries_df.index))
countries_df.head()

In [ ]:
# delete columns 0, 2 to 7
countries2_df = countries_df.drop(countries_df.loc[:, 'sub_region_1':'place_id'].columns, axis = 1)
del countries2_df['country_region_code']
print(len(countries2_df.index))
countries2_df.head()

In [ ]:
# columns after the date column, drop NaN
countries3_df = countries2_df.replace(np.nan, -999)
countries4_df = countries3_df.loc[((countries3_df.retail_and_recreation_percent_change_from_baseline != -999) & (countries3_df.grocery_and_pharmacy_percent_change_from_baseline != -999) & (countries3_df.parks_percent_change_from_baseline != -999) & (countries3_df.transit_stations_percent_change_from_baseline != -999) & (countries3_df.workplaces_percent_change_from_baseline != -999) & (countries3_df.residential_percent_change_from_baseline != -999))]
countries4_df.info()

In [ ]:
# convert percent_from_baseline columns to integer
countries4_df.iloc[:, 2] = countries4_df['retail_and_recreation_percent_change_from_baseline'].astype(int)
countries4_df.iloc[:, 3] = countries4_df['grocery_and_pharmacy_percent_change_from_baseline'].astype(int)
countries4_df.iloc[:, 4] = countries4_df['parks_percent_change_from_baseline'].astype(int)
countries4_df.iloc[:, 5] = countries4_df['transit_stations_percent_change_from_baseline'].astype(int)
countries4_df.iloc[:, 6] = countries4_df['workplaces_percent_change_from_baseline'].astype(int)
countries4_df.iloc[:, 7] = countries4_df['residential_percent_change_from_baseline'].astype(int)
countries4_df.info()

In [ ]:
# rename columns
mobility_final_df = countries4_df.rename(columns = {"country_region":"country",
                                               "retail_and_recreation_percent_change_from_baseline":"retail_and_recreation",
                                               "grocery_and_pharmacy_percent_change_from_baseline":"grocery_and_pharmacy",
                                               "parks_percent_change_from_baseline":"parks",
                                               "transit_stations_percent_change_from_baseline":"transit_stations",
                                               "workplaces_percent_change_from_baseline":"workplaces",
                                               "residential_percent_change_from_baseline":"residential"})

In [ ]:
# set Country as index
mobility_final_df.set_index('country', inplace=True)

In [ ]:
# extract dataframe to csv file
mobility_final_df

# Load dataframe into database

In [ ]:
# create database connection
connection_string = "postgres:postgres@localhost:5432/country_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# confirm tables
engine.table_names()

In [ ]:
# load dataframe into database
mobility_final_df.to_sql(name='mobility', con=engine, if_exists='append', index=True)